In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install h5py
!pip install biosppy
!pip install mtcnn
# !pip install -U keras
!pip install git+https://github.com/JihongJu/keras-resnet3d.git

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 2.3MB 7.8MB/s 
  Cloning https://github.com/JihongJu/keras-resnet3d.git to /tmp/pip-req-build-an43oo7s
  Running command git clone -q https://github.com/JihongJu/keras-resnet3d.git /tmp/pip-req-build-an43oo7s
  Running command git submodule update --init --recursive -q
  Created wheel for keras-resnet3d: filename=keras_resnet3d-0.0.1-cp36-none-any.whl size=4773 sha256=5a33867b031d2c768eb338371c6085021999e3cd673bdfc3b313d7c2c0b4000d
  Stored in directory: /tmp/pip-ephem-wheel-cache-pfg6shra/wheels/98/fa/00/0dfe23f54c8f40a14c53f355effb4b198f7727d6d7f05be8b1
Successfully built keras-resnet3d


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import h5py
import biosppy
from mtcnn.mtcnn import MTCNN
from tqdm import tqdm

Using TensorFlow backend.


In [ ]:
detector = MTCNN()
class FaceAligner:
    def __init__(self, desiredLeftEye=(0.35, 0.35),
        desiredFaceWidth=256, desiredFaceHeight=None):
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth
    def align(self, image):
        result = detector.detect_faces(image)
        leftEyeCenter = result[0]["keypoints"]["left_eye"]
        rightEyeCenter = result[0]["keypoints"]["right_eye"]
        
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))
        
        
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
 
        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist
        
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2,
            (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
 
        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
 
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h),
            flags=cv2.INTER_CUBIC)
        return output

In [ ]:
aligner = FaceAligner(desiredLeftEye=(.25,.25),desiredFaceHeight=400)

base_path="/content/drive/My Drive/cohface/"
X_train_1 = []
X_train_2 = []
# X_train_3 = []
y_train = []

#TODO: Consider the possibility of taking only green channel

def framing(folderImages,window_size=20,stride=10):
    num_images = len(folderImages)
    meta_frames = int(np.floor((num_images-window_size)/stride)) + 1
    output = []
    for i in range(meta_frames):
        imgs=[]
        for j in range(i*stride,window_size+i*stride):
            imgs.append(folderImages[j])
        output.append(imgs)
    return output

def extract_heartrate(signal_path):
    with h5py.File(signal_path,"r") as f:
        pulse,time = list(f['pulse']),list(f['time'])
    ts,filtered,onsets,heart_rate_ts,heart_rate = biosppy.signals.bvp.bvp(pulse,sampling_rate=256,show=False)
    hr = np.mean(heart_rate)
    return hr

def extract_features(video_path,signal_path):
  cap = cv2.VideoCapture(video_path)
  X_video_1 = []
  X_video_2 = []
  X_video_3 = []
  count=0
  numFrames = 100
  while(cap.isOpened()):
    ret, frame = cap.read()
    if(ret and count<numFrames):
      try:
        frame = aligner.align(frame)
        frame = frame/255.0
        frame = np.array(frame,dtype='float16')
        frame[:,:,0] = 0
        frame[:,:,2] = 0
        ROI_1 =frame[:int(frame.shape[0]*0.15),:,:]
        ROI_2 = frame[int(frame.shape[0]*0.3):int(frame.shape[0]*0.45),:,:]
        X_video_1.append(ROI_1)
        X_video_2.append(ROI_2)
        # X_video_3.append(frame)
        count+=1
      except:
        pass
    else:
      break
  if len(X_video_1)==numFrames:
    X_train_1.append(framing(X_video_1))
    X_train_2.append(framing(X_video_2))
    # X_train_3.append(framing(X_video_3))
    heart_rate = extract_heartrate(signal_path)
    # print(heart_rate)
    y_train.append(heart_rate)


#Add a check if data exist dont perform the data preprocessing
for paths in tqdm(sorted(os.listdir(base_path))):
    for folder in sorted(os.listdir(base_path+paths))[:2]:
        signal_path = base_path+paths+'/'+folder + '/data.hdf5'
        video_path = base_path+paths+'/'+folder + '/data.avi'
        extract_features(video_path,signal_path)

X_train_1 = np.array(X_train_1)
np.save('/content/drive/My Drive/Preprocessed_data/X_train_1(forehead_framed_green).npy',X_train_1)
print(X_train_1.shape)
X_train_2 = np.array(X_train_2)
np.save('/content/drive/My Drive/Preprocessed_data/X_train_2(cheeks_framed_green).npy',X_train_2)
print(X_train_2.shape)
# X_train_3 = np.array(X_train_3)
# np.save('/content/drive/My Drive/Preprocessed_data/X_train_3(face_framed).npy',X_train_3)
# print(X_train_3.shape)
y_train = np.array(y_train)
np.save('/content/drive/My Drive/Preprocessed_data/y_train(mean_green).npy',y_train)
print(y_train.shape)

In [ ]:
np.unique(X_train_1[1][0][0][:,:,0])

array([0.], dtype=float16)

In [ ]:
X_train_1 = np.load('/content/drive/My Drive/Preprocessed_data/X_train_1(forehead_framed_green).npy')
print(X_train_1.shape)
X_train_2 = np.load('/content/drive/My Drive/Preprocessed_data/X_train_2(cheeks_framed_green).npy')
print(X_train_2.shape)
y_train = np.load('/content/drive/My Drive/Preprocessed_data/y_train(mean_green).npy')
print(y_train.shape)

(77, 9, 20, 60, 256, 3)
(77, 9, 20, 60, 256, 3)
(77,)


In [ ]:
import keras
from keras import backend as K
from keras.layers import Dense,LSTM,Bidirectional,TimeDistributed,Input,GlobalAveragePooling1D,Flatten,Average,AveragePooling1D,Conv2D,Conv3D,concatenate,MaxPooling2D,GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.utils import plot_model
from keras.optimizers import RMSprop,Adam
from resnet3d import Resnet3DBuilder

In [ ]:
from keras.models import load_model
resnet3dmodel = load_model('/content/drive/My Drive/models/model_normalized.h5')
last_layer = resnet3dmodel.layers[-2]
base_model_1 = Model(inputs = resnet3dmodel.input, outputs=last_layer.output)

for layer in base_model_1.layers:
  layer.trainable = False

base_model_1.summary()

In [ ]:
X_train_1.shape
from sklearn.utils import shuffle
X_train_1,X_train_2,y_train = shuffle(X_train_1,X_train_2,y_train,random_state=0)
X_train_1.shape

(77, 9, 20, 60, 256, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_1,X_test_1,y_train,y_test = train_test_split(X_train_1,y_train,test_size=0.1)

In [ ]:
print(X_train_1.shape)

(69, 9, 20, 60, 256, 3)


In [ ]:
input_shape = X_train_1.shape
#Building ResNet3D 18-layers
resnet3dmodel = Resnet3DBuilder.build_resnet_18((input_shape[-4], input_shape[-3], input_shape[-2], input_shape[-1]), 1)
last_layer = resnet3dmodel.layers[-2]
# dense = Dense(1,activation='relu')(last_layer.output)
base_model_1 = Model(inputs = resnet3dmodel.input, outputs=last_layer.output)
# plot_model(base_model_1,to_file= 'base_model_1.png')
base_model_1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 60, 256,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 10, 30, 128,  65920       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 10, 30, 128,  256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 10, 30, 128,  0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
input_shape = X_train_1.shape
inputs = Input(shape=input_shape[1:])
print(inputs)
resnet = TimeDistributed(base_model_1)(inputs)
print(resnet)
lstm = LSTM(1024,return_sequences=True)(resnet)
print(lstm)
dense = Dense(1)(lstm)
print(dense)
pooling = GlobalAveragePooling1D()(dense)
print(pooling)
base_model = Model(inputs=inputs,outputs=pooling)
plot_model(base_model,to_file= 'model.png')
base_model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error','mean_squared_error'])
base_model.summary()

Tensor("input_2:0", shape=(None, 9, 20, 60, 256, 3), dtype=float32)
Tensor("time_distributed_1/Reshape_1:0", shape=(None, 9, 512), dtype=float32)
Tensor("lstm_1/transpose_1:0", shape=(None, 9, 1024), dtype=float32)
Tensor("dense_2/add:0", shape=(None, 9, 1), dtype=float32)
Tensor("global_average_pooling1d_1/Mean:0", shape=(None, 1), dtype=float32)
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 9, 20, 60, 256, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 9, 512)            33214720  
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 1024)           6295552   
_________________________________________________________________
dense_2 (Dense)              (None, 9, 1)              1025      
_______________________________________

In [ ]:
base_model.fit(X_train_1,y_train.reshape(-1,1),epochs=40,batch_size=6,validation_split=0.1)

Train on 62 samples, validate on 7 samples
Epoch 1/40
62/62 [==============================] - 46s 736ms/step - loss: 49.4081 - mean_absolute_error: 48.5182 - mean_squared_error: 2777.1731 - val_loss: 28.0040 - val_mean_absolute_error: 27.2115 - val_mean_squared_error: 813.1622
Epoch 2/40
62/62 [==============================] - 30s 480ms/step - loss: 26.3596 - mean_absolute_error: 25.6201 - mean_squared_error: 897.3685 - val_loss: 16.7805 - val_mean_absolute_error: 16.0979 - val_mean_squared_error: 332.0302
Epoch 3/40
62/62 [==============================] - 30s 481ms/step - loss: 15.9140 - mean_absolute_error: 15.2601 - mean_squared_error: 434.2666 - val_loss: 9.7617 - val_mean_absolute_error: 9.1423 - val_mean_squared_error: 123.0045
Epoch 4/40
62/62 [==============================] - 30s 485ms/step - loss: 11.4016 - mean_absolute_error: 10.8043 - mean_squared_error: 250.3312 - val_loss: 7.9142 - val_mean_absolute_error: 7.3450 - val_mean_squared_error: 72.7862
Epoch 5/40
62/62 [===

In [ ]:
base_model.save('/content/drive/My Drive/models/forehead_green_0.2')

In [ ]:
print(X_test_1.shape)

(16, 9, 20, 60, 256, 3)


In [ ]:
y_pred = base_model.predict(X_test_1)

In [ ]:
print(y_pred)

[[73.47464]
 [74.52908]
 [74.00823]
 [73.91276]
 [74.47077]
 [74.06731]
 [74.22079]
 [74.02633]]


In [ ]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/models/forehead_green_0.2')
y_pred = model.predict(X_test_1)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
score = r2_score(y_test,y_pred)
RMSE = mean_squared_error(y_test,y_pred,squared=False)
MAE = mean_absolute_error(y_test,y_pred)
print(score)
print(RMSE)
print(MAE)

0.32217835628591474
10.851935361287827
8.47829969577165


In [ ]:
input_1 = Input(shape=input_shape[1:])
input_2 = Input(shape=input_shape[1:])

hr1 = base_model(input_1)
hr2 = base_model(input_2)
hr = Average()([hr1,hr2])

model = Model(inputs=[input_1,input_2],outputs=hr)
model.summary()
plot_model(model,to_file = 'model.png')
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error','mean_squared_error'])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 9, 20, 60, 25 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 9, 20, 60, 25 0                                            
__________________________________________________________________________________________________
model_3 (Model)                 (None, 1)            39511297    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
average_1 (Average)             (None, 1)            0           model_3[1][0]              

In [ ]:
callbacks=[]
model_path = '/content/drive/My Drive/Heart Rate Using Deep Learning/model_1.hdf5'
earlystopping = EarlyStopping(monitor='mean_absolute_error',patience=10,mode='auto',restore_best_weights=True)
# modelcheckpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='mean_absolute_error', mode='auto')
# callbacks.append(modelcheckpoint)
callbacks.append(earlystopping)
model.fit([X_train_1,X_train_2], y_train.reshape(-1,1),epochs=30, batch_size=2, validation_split = 0.2,shuffle=True,callbacks=callbacks)

Train on 61 samples, validate on 16 samples
Epoch 1/30
61/61 [==============================] - 49s 803ms/step - loss: 30.8754 - mean_absolute_error: 30.1090 - mean_squared_error: 1432.8157 - val_loss: 6.6602 - val_mean_absolute_error: 6.0275 - val_mean_squared_error: 54.1590
Epoch 2/30
61/61 [==============================] - 35s 567ms/step - loss: 13.0335 - mean_absolute_error: 12.4673 - mean_squared_error: 284.2149 - val_loss: 6.9229 - val_mean_absolute_error: 6.4216 - val_mean_squared_error: 50.0460
Epoch 3/30
61/61 [==============================] - 34s 565ms/step - loss: 12.1258 - mean_absolute_error: 11.6672 - mean_squared_error: 246.6026 - val_loss: 7.2436 - val_mean_absolute_error: 6.8259 - val_mean_squared_error: 55.3981
Epoch 4/30
61/61 [==============================] - 35s 566ms/step - loss: 12.2622 - mean_absolute_error: 11.8763 - mean_squared_error: 274.5075 - val_loss: 7.2084 - val_mean_absolute_error: 6.8544 - val_mean_squared_error: 58.1263
Epoch 5/30
61/61 [=========

KeyboardInterrupt: ignored